In [ ]:
import os #interacts with the os
import sys #handle system-specific parameters and function
import time #time module to handle time-related tasks like transfer rate etc.
from collections import defaultdict #store and manage packet counts for each IP
from scapy.all import sniff, IP #helps to capture and analyze network packets


#Variable representing the threshhold of the DOS attack
Threshhold = 40
print("THRESHOLD: {Threshhold}")

def packet_callback(packet):
    src_ip = packet[IP].src
    packet_count[src_ip] += 1
    current_time = time.time()
    time_interval = current_time - start_time[0]
    
    if time_interval >= 1:
        for ip, count in packet_count.items():
            packet_rate = count / time_interval
            if packet_rate > Threshhold and ip not in blocked_ips:
                print(f"Blocking IP: {ip}, packet rate : {packet_rate}")
                os.system(f"iptables -A INPUT -s {ip} -j DROP")
                blocked_ips.add(ip)
                
        packet_count.clear()
        start_time[0] = current_time
        
if __name__ == "__main__":
    if os.geteuid() != 0:
        print("This script requires root priviledges.")
        sys.exit(1)
        
        packet_count = defaultdict(int)
        start_time = [time.time()]
        blocked_ips = set()
    print("Monitoring network traffic...")
    sniff(filter="ip", prn=packet_callback, store=0, count=0)

THRESHOLD: {Threshhold}
This script requires root priviledges.


SystemExit: 1